In [7]:
# imports
from scisample.samplers import new_sampler
import plotly.graph_objects as go
from scipy.optimize import minimize
import numpy as np
import yaml
import pandas as pd
import tempfile
import shutil
import os

In [8]:
# functions
def rosenbrock_vector(x):
    """ Rosenbrock function 2D """
    return (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2

def rosenbrock_xy(x, y):
    """ Rosenbrock function 2D """
    return (1 - x)**2 + 100*(y - x**2)**2

def rastrigin(x):
    """ Rastrigin function """
    n = len(x)
    return 10 * n + sum(x**2 - 10 * np.cos(2 * np.pi * x))

def ackley(x):
    """ Ackley function """
    n = len(x)
    return -20 * np.exp(-0.2 * np.sqrt((1/n) * np.sum(x**2))) - \
           np.exp((1/n) * np.sum(np.cos(2 * np.pi * x))) + 20 + np.exp(1)


def yaml_string_to_dict(yaml_string):
    yaml_data = yaml.safe_load(yaml_string)
    return yaml_data

In [9]:
# Initial guess for the minimum
x0 = np.array([0.0, 0.0])

# Use Nelder-Mead algorithm to find the minimum
result = minimize(rosenbrock_vector, x0, method='Nelder-Mead')

# Print the minimum value and corresponding x, y coordinates
print("Minimum value:", result.fun)
print("Minimum point:", result.x)

Minimum value: 3.6861769151759075e-10
Minimum point: [1.00000439 1.00001064]


In [10]:
# Contour plot
x_values = np.linspace(-1, 2, 100)  # Generate x values between -1 and 2
y_values = np.linspace(-1, 2, 100)  # Generate y values between -1 and 2
X, Y = np.meshgrid(x_values, y_values)
Z = np.log(rosenbrock_xy(X, Y))

fig = go.Figure(data=
    go.Contour(
        z=Z,
        x=x_values,
        y=y_values,
        zmin=-2,  # Set z minimum value
        zmax=6,   # Set z maximum value 
    ))
fig.show()

/var/folders/km/vxpqjv0s5ml6xgs2y6bd21yr0007gz/T/ipykernel_51706/2438218603.py:5: RuntimeWarning: divide by zero encountered in log
  Z = np.log(rosenbrock_xy(X, Y))


In [36]:
# Initial set of samples
downselect_ratio = 0.3
yaml_string = """
type: best_candidate
num_samples: 30
# previous_samples: samples.csv # optional
# cost_variable: Z   # required if previous_samples is provided
# downselect_ratio: 0.3 # required if previous_samples is provided
# voxel_overlap: 1.0    # required if previous_samples is provided
parameters:
    X:
        min: -1
        max: 2
    Y:
        min: -1
        max: 2
"""
sample_dictionary = yaml_string_to_dict(yaml_string)

sampler = new_sampler(sample_dictionary)
samples = sampler.get_samples()

for sample in samples:
    sample['Z'] = rosenbrock_xy(sample['X'], sample['Y'])

print("samples:")
df = pd.DataFrame(samples)
# Display the 5 smallest elements of 'Z'
smallest_values = df['Z'].nsmallest(5).to_list()

iterations = 20
print(f"5 smallest {iterations}: {smallest_values}")
while iterations > 0:
    # Create a temporary directory
    temp_dir = tempfile.mkdtemp()
    temp_file = os.path.join(temp_dir, "temp.csv")
    df.to_csv(temp_file)

    # next set of samples
    yaml_string = f"""
    type: best_candidate
    num_samples: 30
    previous_samples: {temp_file} # optional
    cost_variable: Z   # required if previous_samples is provided
    downselect_ratio: {downselect_ratio} # required if previous_samples is provided
    downselect_number: 10 # required if previous_samples is provided
    voxel_overlap: 0.55    # required if previous_samples is provided
    parameters:
        X:
            min: -1
            max: 2
        Y:
            min: -1
            max: 2
    """
    sample_dictionary = yaml_string_to_dict(yaml_string)

    sampler = new_sampler(sample_dictionary)
    samples = sampler.get_samples()

    for sample in samples:
        sample['Z'] = rosenbrock_xy(sample['X'], sample['Y'])

    print("samples:")
    df_new = pd.DataFrame(samples)
    df = pd.concat([df, df_new], ignore_index=True)

    smallest_values = df['Z'].nsmallest(5).to_list()
    print(f"5 smallest {iterations}: {smallest_values}")

    shutil.rmtree(temp_dir)
    iterations -= 1

extrema for new input_labels:  [-0.97707989 -0.99659453] [1.98851581 1.99760855]
down sampling to 30 best candidates from 300 total points.


previous_samples: [1.2023493878679996, 3.593440850206142, 5.852319918937809, 11.110834164972037, 11.450668820797718]


samples:
5 smallest 20: [1.2023493878679996, 3.593440850206142, 5.852319918937809, 11.110834164972037, 11.450668820797718]
extrema for new input_labels:  [-0.91933957 -0.80838938] [1.47172484 1.99420935]
down sampling to 30 best candidates from 297 total points.


previous_samples: [0.0750235142329726, 1.2023493878679996, 1.7874358167154256, 2.0754448451759333, 2.9208879314385]


samples:
5 smallest 20: [0.07502351423297265, 1.2023493878679996, 1.7874358167154256, 2.0754448451759333, 2.9208879314385]
extrema for new input_labels:  [-0.89144936 -0.13358896] [1.32324948 1.85252626]
down sampling to 30 best candidates from 300 total points.


...

samples:
5 smallest 4: [2.1996288197246205e-07, 2.881526985940566e-07, 7.871228236367204e-07, 1.3640026362274277e-06, 1.4175235897648474e-06]
extrema for new input_labels:  [0.99917221 0.        ] [0.99992796 1.00012895]
down sampling to 30 best candidates from 300 total points.


previous_samples: [1.3955143189078972e-08, 2.1996288197246205e-07, 2.274228712293669e-07, 2.679100133506891e-07, 2.881526985940566e-07]


samples:
5 smallest 3: [1.3955143189078972e-08, 2.1996288197246205e-07, 2.274228712293669e-07, 2.6791001335068907e-07, 2.881526985940566e-07]
extrema for new input_labels:  [0.99925481 0.        ] [0.99993163 0.99982776]
down sampling to 30 best candidates from 300 total points.


previous_samples: [1.3955143189078972e-08, 2.9044942413607288e-08, 7.744077406153133e-08, 8.963375980732278e-08, 1.0368890853101539e-07]


samples:
5 smallest 2: [1.3955143189078972e-08, 2.9044942413607288e-08, 7.744077406153132e-08, 8.963375980732277e-08, 1.0368890853101538e-07]
extrema for new input_labels:  [0.99952964 0.        ] [0.9999561  0.99983463]
down sampling to 30 best candidates from 300 total points.
samples:
5 smallest 1: [1.3955143189078972e-08, 2.9044942413607288e-08, 4.188646153366002e-08, 7.744077406153132e-08, 8.539190233921993e-08]


In [37]:
# first random seed (downselect_number = 20, downselect_ratio = 0.3, voxel_overlap 0.6
fig = go.Figure(data=go.Scatter(x=df.index, y=df['Z'], mode='markers'))

# Set y-aZis scale to logarithmic
fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Z vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

fig = go.Figure(data=go.Scatter(x=df.index, y=df['X'], mode='markers'))

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column X vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

fig = go.Figure(data=go.Scatter(x=df.index, y=df['Y'], mode='markers'))

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Y vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

In [35]:
# first random seed (downselect_number = 20, downselect_ratio = 0.3, voxel_overlap 0.6
fig = go.Figure(data=go.Scatter(x=df.index, y=df['Z'], mode='markers'))

# Set y-aZis scale to logarithmic
fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Z vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

fig = go.Figure(data=go.Scatter(x=df.index, y=df['X'], mode='markers'))

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column X vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

fig = go.Figure(data=go.Scatter(x=df.index, y=df['Y'], mode='markers'))

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Y vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

In [33]:
# first random seed (downselect_number = 20, downselect_ratio = 0.3, voxel_overlap 0.6
fig = go.Figure(data=go.Scatter(x=df.index, y=df['Z'], mode='markers'))

# Set y-aZis scale to logarithmic
fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Z vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

fig = go.Figure(data=go.Scatter(x=df.index, y=df['X'], mode='markers'))

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column X vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

fig = go.Figure(data=go.Scatter(x=df.index, y=df['Y'], mode='markers'))

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Y vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

In [31]:
fig = go.Figure(data=go.Scatter(x=df.index, y=df['Z'], mode='markers'))

# Set y-aZis scale to logarithmic
fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Z vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

fig = go.Figure(data=go.Scatter(x=df.index, y=df['X'], mode='markers'))

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column X vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

fig = go.Figure(data=go.Scatter(x=df.index, y=df['Y'], mode='markers'))

# Set y-axis scale to logarithmic
# fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Y vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()

In [7]:
fig = go.Figure(data=go.Scatter(x=df.index, y=df['Z'], mode='markers'))

# Set y-axis scale to logarithmic
fig.update_layout(yaxis_type='log')

# Add labels and title to the plot
fig.update_layout(title=f'Scatter Plot of Column Z vs. Row Index (downselect_ratio = {downselect_ratio})',
                  xaxis_title='Row Index',
                  yaxis_title='Column Z')

# Display the plot
fig.show()